In [ ]:
import folium
import pandas as pd
import branca.colormap as cm
import json
import requests
import io
from shapely.geometry import shape

csv_data = """Rank, Country, Cost of Living Index, Rent Index, Cost of Living Plus Rent Index, Groceries Index, Restaurant Price Index, Local Purchasing Power Index
1, Us Virgin Islands, 98.4, 46.4, 75.4, 106.3, 90.5, 72.3
2, Switzerland, 98.4, 45.2, 74.8, 103.1, 98.4, 159.7
3, Iceland, 83.4, 43.7, 65.9, 90.3, 94.7, 123.5
4, Bahamas, 81.4, 46.9, 66.1, 79.1, 94.5, 58.4
5, Singapore, 79.1, 67.0, 73.7, 70.8, 53.5, 93.8
6, Hong Kong (China), 73.6, 55.7, 65.7, 75.9, 49.7, 101.0
7, Barbados, 70.0, 20.1, 47.9, 78.1, 70.3, 45.9
8, Norway, 69.0, 24.8, 49.4, 71.9, 74.6, 119.5
9, Papua New Guinea, 67.4, 36.6, 53.7, 51.9, 37.9, 14.0
10, Denmark, 66.9, 25.7, 48.6, 61.0, 80.8, 133.6
11, Guernsey, 66.1, 45.7, 57.1, 70.5, 65.9, 136.6
12, Israel, 65.2, 27.1, 48.3, 61.2, 74.5, 110.0
13, United States, 64.9, 40.9, 54.3, 71.7, 69.3, 146.2
14, Luxembourg, 64.8, 42.2, 54.8, 65.3, 72.8, 177.1
15, Australia, 60.9, 29.8, 47.1, 67.9, 59.2, 135.4
16, Austria, 60.7, 22.0, 43.6, 62.2, 59.7, 107.1
17, Netherlands, 60.5, 32.1, 47.9, 56.8, 67.7, 129.9
18, Ireland, 59.8, 39.0, 50.6, 56.9, 65.8, 108.9
19, United Kingdom, 59.2, 30.0, 46.3, 54.8, 65.3, 119.9
20, Puerto Rico, 58.7, 19.9, 41.5, 63.1, 53.4, 111.0
21, Finland, 58.7, 19.1, 41.1, 59.4, 64.7, 124.5
22, Canada, 58.7, 30.3, 46.1, 65.3, 59.9, 109.1
23, Germany, 58.4, 22.4, 42.4, 56.7, 57.5, 127.1
24, France, 58.0, 19.4, 41.0, 63.6, 56.1, 110.0
25, Macao (China), 56.8, 29.4, 44.6, 68.2, 40.4, 103.8
26, South Korea, 56.5, 15.1, 38.2, 74.5, 32.8, 105.0
27, Belgium, 56.5, 19.3, 40.1, 54.6, 67.1, 116.1
28, New Zealand, 55.3, 23.4, 41.2, 62.2, 53.8, 130.5
29, Sweden, 54.2, 19.1, 38.7, 55.5, 56.0, 130.3
30, United Arab Emirates, 54.1, 39.7, 47.8, 42.4, 59.9, 128.3
31, Italy, 51.0, 17.7, 36.3, 53.5, 53.0, 85.1
32, Jamaica, 50.3, 17.7, 35.9, 59.6, 42.8, 36.4
33, Costa Rica, 50.1, 18.5, 36.1, 56.2, 47.8, 50.9
34, Cyprus, 49.5, 24.0, 38.2, 47.0, 52.7, 80.4
35, Estonia, 49.4, 14.1, 33.8, 45.1, 54.0, 84.2
36, Trinidad And Tobago, 48.9, 13.7, 33.3, 54.1, 45.2, 46.1
37, Yemen, 48.4, 5.9, 29.6, 61.5, 33.9, 20.0
38, Bahrain, 48.3, 20.4, 36.0, 46.2, 47.6, 117.5
39, Malta, 48.0, 23.3, 37.1, 48.1, 55.1, 75.3
40, Qatar, 47.5, 34.4, 41.7, 42.2, 46.4, 177.9
41, Greece, 46.5, 11.8, 31.1, 44.4, 51.1, 59.5
42, Uruguay, 46.3, 12.5, 31.3, 46.5, 49.5, 54.1
43, Slovenia, 46.2, 18.9, 34.1, 45.8, 43.9, 83.6
44, Japan, 45.6, 14.1, 31.6, 54.5, 29.4, 119.8
45, Maldives, 45.3, 22.0, 35.0, 50.5, 34.1, 51.9
46, Brunei, 44.4, 15.2, 31.5, 64.6, 29.5, 117.6
47, Taiwan, 44.4, 13.1, 30.5, 58.0, 25.5, 101.2
48, Lithuania, 44.0, 14.4, 30.9, 40.1, 49.1, 79.8
49, Panama, 43.9, 22.5, 34.4, 49.6, 41.9, 46.3
50, Spain, 43.5, 21.2, 33.6, 43.0, 46.1, 98.1
51, Latvia, 43.4, 10.0, 28.6, 40.6, 43.2, 75.8
52, Croatia, 43.2, 14.5, 30.5, 40.9, 46.8, 78.8
53, Ethiopia, 43.2, 15.7, 31.0, 46.5, 22.8, 13.3
54, Czech Republic, 42.6, 17.6, 31.5, 41.0, 35.3, 88.5
55, Slovakia, 42.1, 14.1, 29.7, 43.1, 35.7, 74.0
56, Saudi Arabia, 41.9, 11.5, 28.4, 40.2, 33.9, 146.1
57, Belize, 41.4, 11.6, 28.2, 46.7, 34.7, 69.9
58, Portugal, 41.2, 22.2, 32.8, 40.5, 38.4, 59.7
59, Cuba, 40.8, 11.6, 27.9, 41.1, 26.6, 2.5
60, Kuwait, 40.4, 21.9, 32.2, 33.7, 40.8, 174.0
61, Albania, 39.9, 11.7, 27.4, 40.4, 38.2, 42.7
62, Oman, 39.8, 12.0, 27.5, 38.9, 38.4, 150.3
63, Armenia, 39.6, 17.6, 29.9, 34.5, 42.3, 37.6
64, Palestine, 39.5, 7.8, 25.5, 41.0, 32.8, 48.9
65, Botswana, 39.5, 5.6, 24.5, 28.1, 24.9, 63.5
66, Mozambique, 38.9, 13.2, 27.5, 33.6, 37.1, 25.3
67, Poland, 38.9, 16.1, 28.8, 34.7, 39.5, 91.4
68, Lebanon, 38.8, 13.6, 27.7, 31.2, 40.9, 35.1
69, Ivory Coast, 38.8, 14.5, 28.0, 34.6, 32.2, 13.3
70, Somalia, 38.7, 3.5, 23.1, 30.7, 23.3, 27.7
71, El Salvador, 37.9, 15.6, 28.0, 42.8, 32.4, 33.8
72, Jordan, 37.2, 7.7, 24.1, 35.3, 37.2, 51.7
73, Guatemala, 36.8, 12.6, 26.1, 41.7, 34.8, 38.6
74, Hungary, 36.6, 11.1, 25.3, 35.4, 36.2, 70.7
75, Montenegro, 36.3, 14.9, 26.8, 34.7, 37.7, 56.8
76, Cameroon, 36.2, 17.2, 27.8, 34.7, 42.2, 9.7
77, Serbia, 35.9, 12.2, 25.4, 34.1, 35.5, 57.6
78, Venezuela, 35.9, 5.8, 22.6, 39.5, 39.9, 15.8
79, Argentina, 35.7, 9.9, 24.3, 37.0, 39.8, 41.1
80, Mauritius, 35.6, 9.7, 24.1, 39.1, 29.9, 49.4
81, Bulgaria, 35.4, 10.1, 24.2, 36.7, 36.7, 74.4
82, Chile, 35.1, 10.6, 24.3, 38.1, 36.8, 51.0
83, Zimbabwe, 34.7, 8.5, 23.1, 33.9, 33.2, 29.6
84, Turkey, 34.7, 12.8, 25.0, 35.6, 34.1, 51.2
85, Rwanda, 34.6, 8.2, 22.9, 23.8, 21.2, 20.7
86, Honduras, 34.6, 10.2, 23.8, 39.0, 31.5, 42.5
87, Mexico, 34.5, 14.1, 25.4, 38.2, 36.0, 46.9
88, Dominican Republic, 34.3, 9.8, 23.5, 36.2, 33.3, 31.4
89, Romania, 34.0, 9.6, 23.2, 33.9, 35.7, 73.1
90, Cambodia, 33.9, 9.4, 23.0, 40.9, 24.4, 26.7
91, Zambia, 33.8, 9.8, 23.1, 29.0, 22.1, 23.0
92, Thailand, 33.7, 11.9, 24.1, 40.5, 21.4, 46.0
93, Sri Lanka, 33.5, 6.3, 21.5, 49.9, 23.3, 18.2
94, Fiji, 32.0, 15.6, 24.8, 40.8, 30.4, 56.9
95, Tanzania, 31.8, 8.8, 21.6, 27.1, 22.0, 22.1
96, Bosnia And Herzegovina, 31.6, 6.1, 20.3, 32.9, 24.1, 63.3
97, Moldova, 30.8, 12.6, 22.7, 29.5, 27.2, 46.3
98, China, 30.7, 11.6, 22.2, 34.2, 21.8, 77.9
99, Ghana, 30.6, 12.1, 22.4, 33.3, 30.1, 16.7
100, Mongolia, 30.6, 14.3, 23.4, 38.6, 24.2, 30.8
101, Georgia, 30.4, 12.8, 22.6, 31.2, 33.5, 39.6
102, North Macedonia, 30.3, 6.8, 19.9, 30.2, 24.7, 59.7
103, Ecuador, 30.0, 8.0, 20.3, 33.2, 27.1, 45.6
104, South Africa, 30.0, 11.1, 21.6, 27.5, 29.3, 111.2
105, Malaysia, 29.7, 8.1, 20.1, 37.3, 22.1, 83.8
106, Azerbaijan, 29.5, 9.0, 20.4, 28.3, 33.1, 41.4
107, Peru, 29.4, 9.2, 20.5, 32.4, 25.3, 38.2
108, Namibia, 29.3, 11.4, 21.4, 31.3, 28.9, 62.2
109, Philippines, 28.8, 6.9, 19.1, 34.3, 19.0, 37.3
110, Kenya, 28.2, 7.6, 19.1, 30.2, 25.0, 35.1
111, Morocco, 27.2, 7.1, 18.3, 28.8, 22.2, 41.7
112, Iraq, 26.9, 7.3, 18.2, 27.9, 22.9, 54.5
113, Vietnam, 26.6, 9.2, 18.9, 32.1, 16.8, 42.3
114, Kazakhstan, 26.6, 10.5, 19.4, 27.4, 28.0, 50.8
115, Nigeria, 26.4, 21.8, 24.4, 31.8, 21.6, 10.6
116, Uganda, 26.0, 8.5, 18.2, 28.3, 24.7, 16.4
117, Colombia, 26.0, 8.1, 18.1, 27.7, 22.9, 36.0
118, Kosovo (Disputed Territory), 25.7, 6.7, 17.3, 27.1, 20.9, 52.9
119, Brazil, 25.6, 7.2, 17.4, 25.9, 22.5, 42.3
120, Tunisia, 25.5, 4.6, 16.3, 31.0, 16.1, 34.7
121, Kyrgyzstan, 25.4, 10.8, 18.9, 28.0, 21.4, 35.6
122, Bolivia, 25.2, 8.3, 17.7, 25.9, 22.4, 43.9
123, Belarus, 25.0, 8.7, 17.8, 24.7, 27.5, 54.7
124, Indonesia, 24.6, 7.7, 17.1, 32.5, 15.2, 33.9
125, Ukraine, 24.6, 7.8, 17.1, 25.2, 21.9, 46.3
126, Algeria, 24.5, 3.4, 15.2, 33.2, 14.5, 34.5
127, Uzbekistan, 24.2, 12.0, 18.8, 26.6, 20.8, 44.5
128, Iran, 23.6, 8.5, 16.9, 20.8, 16.9, 28.4
129, Syria, 23.3, 3.5, 14.5, 24.5, 17.8, 5.4
130, Paraguay, 23.0, 8.6, 16.6, 22.7, 19.8, 41.0
131, Nepal, 23.0, 3.0, 14.1, 24.0, 18.7, 29.0
132, Russia, 22.3, 7.5, 15.8, 21.8, 21.8, 56.2
133, Bangladesh, 20.9, 2.4, 12.7, 25.2, 15.4, 35.6
134, Madagascar, 20.7, 4.5, 13.5, 21.6, 14.3, 15.0
135, Afghanistan, 19.9, 2.0, 12.0, 17.3, 14.3, 35.0
136, India, 19.5, 5.0, 13.1, 22.5, 16.6, 82.2
137, Egypt, 19.0, 3.5, 12.2, 19.7, 18.2, 21.1
138, Libya, 18.6, 3.9, 12.1, 22.8, 13.9, 51.6
139, Pakistan, 17.8, 2.9, 11.2, 17.2, 15.0, 30.5"""

# Convert the CSV string to a DataFrame
cost_df = pd.read_csv(io.StringIO(csv_data))

# Clean column names
cost_df.columns = cost_df.columns.str.strip()

# Create base map centered on a global view
world_map = folium.Map(
    location=[20, 0],
    zoom_start=2,
    tiles="OpenStreetMap", 
    scrollWheelZoom=True,
    control_scale=True,
    width='100%',
    height='100%'
)

# Define the GeoJSON URL
geojson_url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/world-countries.json'

# Download the GeoJSON data
geo_data = requests.get(geojson_url).json()

# Define metrics with display names (in desired order)
metrics = [
    ('Cost of Living Index', 'Cost of Living Index'),
    ('Rank', 'Cost of Living Rank'),
    ('Rent Index', 'Rent Index'),
    ('Cost of Living Plus Rent Index', 'Cost of Living + Rent'),
    ('Groceries Index', 'Groceries Index'),
    ('Restaurant Price Index', 'Restaurant Prices'),
    ('Local Purchasing Power Index', 'Purchasing Power')
]

# Define color scales for each metric
color_scales = {
    'Rank': 'PuRd_r',             # Reversed - lower rank is better
    'Cost of Living Index': 'OrRd',
    'Rent Index': 'YlOrBr',
    'Cost of Living Plus Rent Index': 'OrRd',
    'Groceries Index': 'YlOrRd',
    'Restaurant Price Index': 'YlOrBr',
    'Local Purchasing Power Index': 'Greens'  # Higher is better
}

# Create a mapping dictionary for country name standardization
country_name_map = {
    'United States': 'United States of America',
    'Russia': 'Russian Federation',
    'South Korea': 'Korea, Republic of',
    'Iran': 'Iran, Islamic Republic of',
    'Vietnam': 'Viet Nam',
    'Venezuela': 'Venezuela, Bolivarian Republic of',
    'Tanzania': 'Tanzania, United Republic of',
    'Syria': 'Syrian Arab Republic',
    'Moldova': 'Moldova, Republic of',
    'Taiwan': 'Taiwan, Province of China',
    'Bolivia': 'Bolivia, Plurinational State of',
    'Ivory Coast': "Côte d'Ivoire",
    'North Macedonia': 'Macedonia, the former Yugoslav Republic of',
    'Libya': 'Libyan Arab Jamahiriya',
    'Bosnia And Herzegovina': 'Bosnia and Herzegovina',
    'Brunei': 'Brunei Darussalam',
    'United Kingdom': 'United Kingdom',
    'Us Virgin Islands': 'Virgin Islands, U.S.',
    'Hong Kong (China)': 'Hong Kong',
    'Macao (China)': 'Macao',
    'China': 'China',
    'Kosovo (Disputed Territory)': 'Kosovo',
    'Trinidad And Tobago': 'Trinidad and Tobago',
    'United Arab Emirates': 'United Arab Emirates',
    'Dominican Republic': 'Dominican Republic',
    'Czech Republic': 'Czech Republic',
    'New Zealand': 'New Zealand',
    'Costa Rica': 'Costa Rica',
    'El Salvador': 'El Salvador',
    'Saudi Arabia': 'Saudi Arabia',
    'South Africa': 'South Africa',
    'Papua New Guinea': 'Papua New Guinea',
    'Netherlands': 'Netherlands'
}

# Standardize country names in the DataFrame
for index, row in cost_df.iterrows():
    country = row['Country'].strip()
    if country in country_name_map:
        cost_df.at[index, 'Country_Mapped'] = country_name_map[country]
    else:
        cost_df.at[index, 'Country_Mapped'] = country

# Create mapping dict for tooltips
country_data = {}
for _, row in cost_df.iterrows():
    try:
        country_data[row['Country_Mapped']] = {
            'Rank': int(row['Rank']),
            'Cost of Living Index': float(row['Cost of Living Index']),
            'Rent Index': float(row['Rent Index']),
            'Cost of Living Plus Rent Index': float(row['Cost of Living Plus Rent Index']),
            'Groceries Index': float(row['Groceries Index']),
            'Restaurant Price Index': float(row['Restaurant Price Index']),
            'Local Purchasing Power Index': float(row['Local Purchasing Power Index'])
        }
    except:
        pass  # Skip if conversion fails

# English display names for some countries that might have non-English names
english_country_names = {
    'Deutschland': 'Germany',
    'España': 'Spain',
    'Россия': 'Russia',
    'Polska': 'Poland',
    'Česko': 'Czech Republic',
    'Österreich': 'Austria',
    'Schweiz': 'Switzerland',
    'Sverige': 'Sweden',
    'Suomi': 'Finland',
    'Norge': 'Norway',
    'Danmark': 'Denmark',
    'Nederland': 'Netherlands',
    'Belgique': 'Belgium',
    'Ελλάδα': 'Greece',
    'България': 'Bulgaria',
    'România': 'Romania',
    'Magyarország': 'Hungary',
    'Slovensko': 'Slovakia',
    'Hrvatska': 'Croatia',
    'Србија': 'Serbia',
    'Ísland': 'Iceland',
    'Slovenija': 'Slovenia',
    'Latvija': 'Latvia',
    'Eesti': 'Estonia',
    'Lietuva': 'Lithuania',
    'Κύπρος': 'Cyprus',
    'Lëtzebuerg': 'Luxembourg',
    'Malta': 'Malta',
    '日本': 'Japan',
    '中国': 'China',
    '대한민국': 'South Korea',
    'भारत': 'India',
    'ประเทศไทย': 'Thailand',
    'Việt Nam': 'Vietnam',
    'Indonesia': 'Indonesia',
    'Россия': 'Russia',
    'Украина': 'Ukraine',
    'Беларусь': 'Belarus',
    'Brasil': 'Brazil',
    'México': 'Mexico',
    'Argentina': 'Argentina',
    'Chile': 'Chile',
    'Perú': 'Peru',
    'Colombia': 'Colombia',
    'Venezuela': 'Venezuela',
    'Türkiye': 'Turkey',
    'مصر': 'Egypt',
    'المغرب': 'Morocco',
    'الجزائر': 'Algeria',
    'تونس': 'Tunisia',
    'ليبيا': 'Libya',
    'السعودية': 'Saudi Arabia',
    'الإمارات': 'UAE',
    'قطر': 'Qatar',
    'الكويت': 'Kuwait',
    'عمان': 'Oman',
    'البحرين': 'Bahrain',
    'لبنان': 'Lebanon',
    'الأردن': 'Jordan',
    'سوريا': 'Syria',
    'العراق': 'Iraq',
    'إيران': 'Iran'
}

# Default to show Cost of Living Index first
default_metric = 'Cost of Living Index'
default_title = 'Cost of Living Index'

# Create a container for the colorbar
colorbar_div = folium.Element("""
<div id="dynamic-colorbar" style="position:fixed; left:50px; top:10px; z-index:9999; background:white; padding:5px; border-radius:5px; box-shadow: 0 0 10px rgba(0,0,0,0.2);"></div>
""")
world_map.get_root().html.add_child(colorbar_div)

# Create colormaps for each metric
colormaps = {}
colorbar_htmls = {}

for col, title in metrics:
    # Get min/max values for better color distribution
    vmin = cost_df[col].min()
    vmax = cost_df[col].max()
    
    # Define quantiles for better color distribution
    q = list(cost_df[col].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    
    # Create a custom colormap with fixed steps
    if col == 'Rank' or col == 'Local Purchasing Power Index':
        # For metrics where lower is worse (higher is better)
        colormap = cm.LinearColormap(
            colors=['#fee5d9', '#fcbba1', '#fc9272', '#fb6a4a', '#de2d26', '#a50f15'], 
            index=q,
            vmin=vmin,
            vmax=vmax,
            caption=title
        )
    else:
        # For metrics where higher is worse
        colormap = cm.LinearColormap(
            colors=['#ffffcc', '#ffeda0', '#fed976', '#feb24c', '#fd8d3c', '#f03b20', '#bd0026'], 
            index=q,
            vmin=vmin,
            vmax=vmax,
            caption=title
        )
    
    colormaps[col] = colormap
    # Store colorbar HTML for this metric
    colorbar_htmls[title] = colormap._repr_html_()

# Create a feature group for each metric
feature_groups = {}
for col, title in metrics:
    # Create a feature group
    fg = folium.FeatureGroup(name=title, show=(title == default_title))
    feature_groups[title] = fg
    
    # Define a style function for this metric
    def style_function(feature, col=col, colormap=colormaps[col]):
        country_name = feature['properties']['name']
        if country_name in country_data and col in country_data[country_name]:
            return {
                'fillColor': colormap(country_data[country_name][col]),
                'color': 'black',
                'weight': 0.5,
                'fillOpacity': 0.7
            }
        else:
            return {'fillColor': '#f0f0f0', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.3}

    # Create a tooltip function for this layer and metric
    def tooltip_function(feature, col=col, title=title):
        country_name = feature['properties']['name']
        
        # Use English name if available
        display_name = english_country_names.get(country_name, country_name)
        
        if country_name in country_data:
            data = country_data[country_name]
            
            # Handle metric value formatting
            if col in data:
                metric_value = f"{data[col]:.1f}"
            else:
                metric_value = "N/A"
                
            popup_content = f"""
            <div style="min-width:200px">
                <h4 style="margin:0 0 5px;padding-bottom:5px;border-bottom:1px solid #ccc; font-weight:bold; font-size:16px;">{display_name}</h4>
                <table style="width:100%;border-collapse:collapse;font-size:13px; margin-top:5px;">
                    <tr><td><b>Rank:</b></td><td align="right">{data['Rank']}</td></tr>
                    <tr><td><b>Cost of Living:</b></td><td align="right">{data['Cost of Living Index']:.1f}</td></tr>
                    <tr><td><b>Rent:</b></td><td align="right">{data['Rent Index']:.1f}</td></tr>
                    <tr><td><b>Groceries:</b></td><td align="right">{data['Groceries Index']:.1f}</td></tr>
                    <tr><td><b>Restaurant:</b></td><td align="right">{data['Restaurant Price Index']:.1f}</td></tr>
                    <tr><td><b>Purchasing Power:</b></td><td align="right">{data['Local Purchasing Power Index']:.1f}</td></tr>
                </table>
                <div style="margin-top:8px;font-weight:bold;color:#c00; font-size:14px;">
                    Current View: {title} - {metric_value}
                </div>
            </div>
            """
            return popup_content
        else:
            return f"<b>{display_name}</b><br>No data available"

    # Add the GeoJSON layer with tooltips for this metric
    folium.GeoJson(
        geo_data,
        name=title,
        style_function=style_function,
        highlight_function=lambda x: {'weight': 2, 'color': '#333333', 'fillOpacity': 0.8}
    ).add_to(fg)
    
    # Now add custom tooltips and country labels
    for feature in geo_data['features']:
        country_name = feature['properties']['name']
        display_name = english_country_names.get(country_name, country_name)
        tooltip_html = tooltip_function(feature)
        
        # Create a tooltip layer for this feature
        folium.features.GeoJson(
            {"type": "Feature", "geometry": feature["geometry"], "properties": feature["properties"]},
            name=f"tooltip_{display_name}",
            style_function=lambda x: {"fillOpacity": 0.0, "weight": 0},
            tooltip=folium.Tooltip(tooltip_html, sticky=True)
        ).add_to(fg)
        
        # Add country labels to major countries
        if country_name in country_data:
            # Get the center of the country for label placement
            country_geom = shape(feature['geometry'])
            center = list(country_geom.centroid.coords)[0]
            # Only add labels for countries with area above threshold (to avoid cluttering)
            if country_geom.area > 0.5:  # Threshold can be adjusted
                folium.map.Marker(
                    location=[center[1], center[0]],  # Folium uses lat, lon
                    icon=folium.DivIcon(
                        icon_size=(100, 20),
                        icon_anchor=(50, 10),
                        html=f'<div style="font-size: 10px; color: #333; font-weight: bold; text-shadow: 1px 1px 1px #fff, -1px -1px 1px #fff, 1px -1px 1px #fff, -1px 1px 1px #fff;">{display_name}</div>'
                    )
                ).add_to(fg)
    
    # Add this feature group to the map
    fg.add_to(world_map)

# Add layer control to toggle between metrics
folium.LayerControl(collapsed=False).add_to(world_map)

# Create a shorter header for the map
header_html = """
<div style="position:fixed; top:10px; left:50%; transform:translateX(-50%); 
    z-index:9999; background:white; padding:5px 10px; border-radius:5px; 
    font-family:Arial; font-weight:bold; font-size:16px; 
    box-shadow: 0 0 10px rgba(0,0,0,0.2);">
    Global Cost of Living
</div>
"""
world_map.get_root().html.add_child(folium.Element(header_html))

# Add JavaScript to handle layer switching and dynamic colorbar
js_code = """
<script>
(function() {
    // Store all colorbar HTMLs
    var colorbarHTMLs = {
"""

# Add each metric's colorbar HTML as a JavaScript string
for i, (_, title) in enumerate(metrics):
    if i > 0:
        js_code += ",\n"
    # Properly escape the HTML for JavaScript
    html_escaped = colorbar_htmls[title].replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')
    js_code += f"        '{title}': '{html_escaped}'"

js_code += """
    };
    
    // Function to update the colorbar
    function updateColorbar(layerName) {
        var colorbarDiv = document.getElementById('dynamic-colorbar');
        if (colorbarHTMLs[layerName]) {
            colorbarDiv.innerHTML = colorbarHTMLs[layerName];
        }
    }
    
    // Set default colorbar
    setTimeout(function() {
        // Set the initial colorbar
        updateColorbar('""" + default_title + """');
        
        // Add event listeners to layer control checkboxes after they're fully loaded
        var checkInterval = setInterval(function() {
            var layerControls = document.querySelectorAll('.leaflet-control-layers-overlays input[type="checkbox"]');
            if (layerControls.length > 0) {
                clearInterval(checkInterval);
                
                layerControls.forEach(function(checkbox) {
                    checkbox.addEventListener('change', function() {
                        if (this.checked) {
                            // Get the layer name from the next element (the label)
                            var layerName = this.nextElementSibling.textContent.trim();
                            updateColorbar(layerName);
                            
                            // Uncheck all other checkboxes
                            layerControls.forEach(function(cb) {
                                if (cb !== checkbox && cb.checked) {
                                    cb.checked = false;
                                }
                            });
                        }
                    });
                });
            }
        }, 200);
    }, 1000);
})();
</script>
"""

world_map.get_root().html.add_child(folium.Element(js_code))

# Apply CSS to make the map fill the container
css = """
<style>
.folium-map {
    width: 100%;
    height: 100vh;
    position: absolute;
    top: 0;
    left: 0;
}
</style>
"""
world_map.get_root().html.add_child(folium.Element(css))

# Save the map as an HTML file for Jekyll
world_map.save('global_cost_of_living_map.html')

# Display the map
world_map

NameError: name 'shape' is not defined